In [1]:
import meshplot as mp
import numpy as np
from skimage.measure import marching_cubes
from preprocess.util import Interpolation
import networkx as nx

In [2]:
def plotCenterline( points, point_size=0.1 ):
    p = mp.plot( np.array([points[0]]) , shading={'point_color': 'red', 'point_size':point_size },return_plot=True)

    for i in range(1, len(points)):
        p.add_points( np.array([points[i]]), shading={'point_color': 'red', 'point_size':point_size } )
        p.add_lines( np.array([points[i - 1]]), np.array([points[i]]))    

    return p

In [3]:
centerline = nx.Graph()
centerline.add_node(0, position=np.array([0,0,0]))
centerline.add_node(1, position=np.array([0,0.3,0]))
centerline.add_node(2, position=np.array([0,0.7,0.1]))
centerline.add_node(3, position=np.array([0,1,0.2]))

centerline.add_edges_from([(0,1), (1,2)])

In [4]:
p = plotCenterline(np.array([ data['position'] for node, data in centerline.nodes(data=True)]))

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

La sdf del cilindro, normalizado entre 0 y 1

In [5]:
def sdfSphere( p ):
    return np.linalg.norm(p) - 1


def voxels( samples=30):
    grid = []

    for x in np.linspace(-1, 1, samples):
        plano = []
        for y in np.linspace(-1, 1, samples):
            row = []
            for z in np.linspace(-1, 1, samples):
                row.append( sdfSphere(np.array([x,y,z])) )
            plano.append(row)
        grid.append(plano)

    grid = np.array(grid)
    return grid
    
def mesh( voxels ):
    verts, faces, normals, values = marching_cubes( voxels, 0)
    return verts, faces

In [6]:
verts, faces = mesh( voxels())
mp.plot( verts, faces)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(14.499999…

In [7]:
class sdfUnion:
    def __init__( self, positions ):
        self.pos = positions
        self.curve = Interpolation( positions )
        self.bases = self.curve.basisAlong( [0, 0.3, 0.7, 1] )
        
    def __call__( self, p ):
        pInBasis = self._toBasis( p )

        return np.min( [self.sdfSphere(p_b) for p_b in pInBasis ] )

    @staticmethod
    def sdfSphere( p ):
        return np.linalg.norm(p) - 0.1

    def _toBasis( self, p ):
        return [ np.linalg.solve( base.T , p - center) for base, center in zip(self.bases, self.pos) ]



In [8]:
union = sdfUnion(np.array([ data['position'] for _,data in centerline.nodes(data=True) ]))


def voxels2( samples=30):
    grid = []

    for x in np.linspace(-1, 1, samples):
        plano = []
        for y in np.linspace(-1, 1, samples):
            row = []
            for z in np.linspace(-1, 1, samples):
                row.append( union(np.array([x,y,z])) )
            plano.append(row)
        grid.append(plano)

    grid = np.array(grid)
    return grid
    


In [9]:
import time

startV = time.time()
grid = voxels2( samples = 50 )
endV = time.time()

startM = time.time()
verts, faces = mesh( grid )
endM = time.time()

print( endV- startV, endM - startM)
mp.plot( verts, faces)

4.28408145904541 0.001192331314086914


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(24.5, 35.…